In [1]:
import numpy as np
import pandas as pd
import networkx as nx

from sklearn.cluster import KMeans
from sklearn import manifold

from scipy.optimize import minimize
from scipy.spatial.distance import pdist, squareform

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
orderdf = pd.read_csv('ignored_assets/orders.csv', parse_dates = [6,7])


In [3]:
coredf = orderdf[orderdf['dest_district_hash'].isin(orderdf['start_district_hash'].unique())].copy()


In [4]:
# Focus on a given date
date_range=pd.Period('2016-01-01','D')
coredf['dayts']=coredf['timestamp'].dt.to_period('D')


In [5]:
# For some reason this, of all things, takes forever. Input on how to make this faster is very welcome
myorders = coredf[coredf['dayts']==date_range].copy()

In [7]:
myorders=myorders.sort_values(by='timestamp')

In [8]:
posdf=pd.read_csv('inferred_locations.csv')
posdf=posdf.set_index('start_district_hash')

In [9]:
myorders['month']=myorders['timestamp'].dt.month
myorders['day']=myorders['timestamp'].dt.day
myorders['hour']=myorders['timestamp'].dt.hour
myorders['minute']=myorders['timestamp'].dt.minute
myorders['second']=myorders['timestamp'].dt.second

In [10]:
myorders['start_x']=posdf.loc[myorders['start_district_hash'][:]].x.values
myorders['start_y']=posdf.loc[myorders['start_district_hash'][:]].y.values
myorders['dest_x']=posdf.loc[myorders['dest_district_hash'][:]].x.values
myorders['dest_y']=posdf.loc[myorders['dest_district_hash'][:]].y.values

In [11]:
MATLABorders=myorders[['month', 'day','hour','minute','second','start_x','start_y','dest_x','dest_y']]

In [14]:
MATLABorders.to_csv('ignored_assets/MATLAB_orders.csv')

In [12]:
MATLABorders

,month,day,hour,minute,second,start_x,start_y,dest_x,dest_y
262258,1,1,0,0,3,-7.068535,-9.644402,-20.130231,-18.136877
210341,1,1,0,0,4,-7.068535,-9.644402,-6.465359,3.205928
231406,1,1,0,0,4,-6.465359,3.205928,-1.706239,-4.296803
268247,1,1,0,0,5,-7.079729,-3.819357,-7.079729,-3.819357
323063,1,1,0,0,5,-6.465359,3.205928,-13.038269,-10.489150
188962,1,1,0,0,5,-1.535667,-23.601203,5.448162,-17.451462
150247,1,1,0,0,6,0.376204,3.572515,0.376204,3.572515
411831,1,1,0,0,6,92.009561,-30.439121,92.009561,-30.439121
146611,1,1,0,0,7,-21.486385,28.765957,-12.463415,28.757639
195244,1,1,0,0,7,13.452524,19.127030,6.142620,22.889402


In [ ]:
myorders